In [1]:
#%pip install rasterio

In [2]:
import boto3
import rasterio
from rasterio.plot import show_hist
from rasterio.plot import show
from rasterio.windows import Window
import sagemaker
from sagemaker import get_execution_role

In [3]:
# Set session and s3 connection objects
sess = sagemaker.Session()
role = get_execution_role()
conn = boto3.client("s3")
bucket = "w210-poverty-mapper"

In [4]:
zone_list = ["42S", "42T", 
             "43S", 
             "44R", 
             "45Q", "45R", 
             "46Q", "46R", 
             "50N", "50P", "50Q", 
             "51L", "51N", "51P", "51Q", 
             "52L", "52N", "52P"]

# Set image size
size = 224

In [19]:
# Create dictionary with zones as keys and empty lists as values
partial_tifs = {new_list: [] for new_list in zone_list}
#partial_tifs

# Find tifs that will create partial tiles
for zone in zone_list:
    contents = conn.list_objects(
        Bucket=bucket, 
        Prefix="sentinel2_composite/raw_data/" + zone)["Contents"]

    for file in contents:
        if file["Key"].endswith(".tif"):
            #print("Processing {}...".format(file["Key"]))
    
            tif = rasterio.open("s3://w210-poverty-mapper/" + file["Key"])
            width = tif.width
            height = tif.height
            
            if tif.width % size != 0 or tif.height % size !=0:
                partial_tifs[zone].append(file["Key"])

In [20]:
partial_tifs

{'42S': ['sentinel2_composite/raw_data/42S/S2_percentile_30_UTM_341-0000000000-0000046592.tif',
  'sentinel2_composite/raw_data/42S/S2_percentile_30_UTM_341-0000023296-0000046592.tif',
  'sentinel2_composite/raw_data/42S/S2_percentile_30_UTM_341-0000046592-0000046592.tif',
  'sentinel2_composite/raw_data/42S/S2_percentile_30_UTM_341-0000069888-0000000000.tif',
  'sentinel2_composite/raw_data/42S/S2_percentile_30_UTM_341-0000069888-0000023296.tif',
  'sentinel2_composite/raw_data/42S/S2_percentile_30_UTM_341-0000069888-0000046592.tif'],
 '42T': ['sentinel2_composite/raw_data/42T/S2_percentile_30_UTM_281-0000000000-0000046592.tif',
  'sentinel2_composite/raw_data/42T/S2_percentile_30_UTM_281-0000023296-0000046592.tif',
  'sentinel2_composite/raw_data/42T/S2_percentile_30_UTM_281-0000046592-0000046592.tif',
  'sentinel2_composite/raw_data/42T/S2_percentile_30_UTM_281-0000069888-0000000000.tif',
  'sentinel2_composite/raw_data/42T/S2_percentile_30_UTM_281-0000069888-0000023296.tif',
  'sen

In [30]:
path = "sentinel2_composite/raw_data/50N/S2_percentile_30_UTM_589-0000023296-0000046592.tif"
check_tif = rasterio.open("s3://w210-poverty-mapper/" + path)

print("width: {}".format(check_tif.width % size))
print("height: {}".format(check_tif.height % size))

width: 44
height: 0
